
# Siamese Keystroke Authentication

This notebook trains a **Siamese MLP** for **fixed‑text keystroke authentication** and evaluates it in a way that matches the **LightGBM paper's binary setup**:

- **Seen users only** for now
- **Per‑user binary task authentication**: *user u* vs *all others*
- **70/30 split** of samples **per user**
- Report **per‑user Precision/Recall/F1/FAR/FRR** and their **macro means**


## 1) Config

In [ ]:
DATA_PATH = "/content/fixed-text.csv"
SEED = 1337
USE_CUDA = True

TRAIN_FRACTION_PER_USER = 0.70
EPOCHS = 30
BATCH_SIZE = 256
LEARNING_RATE = 1e-3

MAX_POS_PAIRS_PER_USER = 200
NEGATIVES_PER_POSITIVE = 2

EMBED_DIM = 128
DROPOUT_P = 0.10
MARGIN = 1.2

print("Config set.")

Config set.


## 2) Imports & device

In [ ]:
import os, re, numpy as np, pandas as pd
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support

np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = "cuda" if (USE_CUDA and torch.cuda.is_available()) else "cpu"
print("Device:", DEVICE)

Device: cuda


## 3) Load data → keep digraphs → 70/30 split → standardize

In [ ]:
raw = pd.read_csv(DATA_PATH)
digraph_cols = [c for c in raw.columns if re.match(r'^[DU][DU]\.[^.]+\.[^.]+$', c)]
df = raw[['participant','repetition'] + digraph_cols].dropna(subset=digraph_cols).copy()
df['participant'] = df['participant'].astype(str)

# split by repetition per user
train_parts, test_parts = [], []
for uid, g in df.groupby('participant'):
    g = g.sort_values('repetition')
    k = int(round(TRAIN_FRACTION_PER_USER * len(g)))
    train_parts.append(g.iloc[:k]); test_parts.append(g.iloc[k:])
train_df = pd.concat(train_parts, ignore_index=True)
test_df  = pd.concat(test_parts,  ignore_index=True)

# scale on train, apply to test
scaler = StandardScaler()
train_df.loc[:, digraph_cols] = scaler.fit_transform(train_df[digraph_cols].astype('float32'))
test_df.loc[:,  digraph_cols] = scaler.transform(   test_df[digraph_cols].astype('float32'))

print(f"Users: {df['participant'].nunique()} | Train rows: {len(train_df)} | Test rows: {len(test_df)} | Features: {len(digraph_cols)}")

Users: 99 | Train rows: 13839 | Test rows: 5933 | Features: 45


## 4) Pair dataset (compact)

In [ ]:
class PairDataset(Dataset):
    """Balanced positive/negative pairs for Siamese training (rebuilt each epoch)."""
    def __init__(self, frame, feature_cols, max_pos_per_user=200, neg_per_pos=2, seed=SEED):
        rng = np.random.RandomState(seed)
        self.pairs = []
        groups = {u: g[feature_cols].values.astype('float32') for u, g in frame.groupby('participant')}
        users = list(groups.keys())

        # positives
        for u in users:
            X = groups[u]; n = len(X)
            if n < 2: continue
            target = min(max_pos_per_user, n*(n-1)//2)
            made, seen = 0, set()
            while made < target:
                i, j = rng.randint(0, n), rng.randint(0, n)
                if i==j: continue
                key = (i,j) if i<j else (j,i)
                if key in seen: continue
                seen.add(key); self.pairs.append((X[i], X[j], 1)); made += 1

        # negatives
        total_pos = len(self.pairs)
        if len(users) >= 2:
            for _ in range(total_pos * neg_per_pos):
                ua, ub = rng.choice(users, size=2, replace=True)
                while ub == ua and len(users) > 1:
                    ub = rng.choice(users)
                Xa, Xb = groups[ua], groups[ub]
                ia, ib = rng.randint(0, len(Xa)), rng.randint(0, len(Xb))
                self.pairs.append((Xa[ia], Xb[ib], 0))

        rng.shuffle(self.pairs)

    def __len__(self): return len(self.pairs)
    def __getitem__(self, i):
        xa, xb, y = self.pairs[i]
        return torch.from_numpy(xa), torch.from_numpy(xb), torch.tensor([y], dtype=torch.float32)

## 5) Siamese MLP (LayerNorm) + contrastive loss

In [ ]:
class SiameseMLP(nn.Module):
    def __init__(self, d_in, d_emb=128, p_drop=0.10):
        super().__init__()
        self.f = nn.Sequential(
            nn.Linear(d_in, 256), nn.ReLU(), nn.LayerNorm(256), nn.Dropout(p_drop),
            nn.Linear(256, 128), nn.ReLU(), nn.LayerNorm(128),
            nn.Linear(128, d_emb)
        )
    def embed(self, x):
        z = self.f(x); return F.normalize(z, p=2, dim=1)
    def forward(self, xa, xb): return self.embed(xa), self.embed(xb)

def contrastive_loss(za, zb, y, margin=1.2):
    d = torch.norm(za - zb, dim=1)
    return (y.view(-1)*(d**2) + (1-y.view(-1))*F.relu(margin-d)**2).mean()

## 6) Train (fixed epochs, no validation)

In [ ]:
model = SiameseMLP(len(digraph_cols), d_emb=EMBED_DIM, p_drop=DROPOUT_P).to(DEVICE)
opt = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for ep in range(1, EPOCHS+1):
    ds = PairDataset(train_df, digraph_cols, MAX_POS_PAIRS_PER_USER, NEGATIVES_PER_POSITIVE, seed=np.random.randint(1_000_000))
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
    model.train(); total, steps = 0.0, 0
    for xa, xb, y in loader:
        xa, xb, y = xa.to(DEVICE), xb.to(DEVICE), y.to(DEVICE)
        opt.zero_grad()
        za, zb = model(xa, xb)
        loss = contrastive_loss(za, zb, y, margin=MARGIN)
        loss.backward(); opt.step()
        total += loss.item(); steps += 1
    print(f"Epoch {ep:02d}  loss={total/max(1,steps):.4f}")

Epoch 01  loss=0.2152
Epoch 02  loss=0.1925
Epoch 03  loss=0.1861
Epoch 04  loss=0.1831
Epoch 05  loss=0.1774
Epoch 06  loss=0.1748
Epoch 07  loss=0.1715
Epoch 08  loss=0.1680
Epoch 09  loss=0.1652
Epoch 10  loss=0.1634
Epoch 11  loss=0.1602
Epoch 12  loss=0.1580
Epoch 13  loss=0.1568
Epoch 14  loss=0.1563
Epoch 15  loss=0.1532
Epoch 16  loss=0.1527
Epoch 17  loss=0.1499
Epoch 18  loss=0.1485
Epoch 19  loss=0.1472
Epoch 20  loss=0.1462
Epoch 21  loss=0.1436
Epoch 22  loss=0.1431
Epoch 23  loss=0.1412
Epoch 24  loss=0.1405
Epoch 25  loss=0.1400
Epoch 26  loss=0.1370
Epoch 27  loss=0.1373
Epoch 28  loss=0.1370
Epoch 29  loss=0.1351
Epoch 30  loss=0.1336


## 7) Evaluate per-user binary metrics (macro means)

In [ ]:
def evaluate_per_user_binary(model, train_df, test_df, feature_cols):
    model.eval(); rng = np.random.RandomState(SEED+7)

    # embed test samples for each user once
    test_embeds = {}
    with torch.no_grad():
        for uid, sub in test_df.groupby('participant'):
            X = sub[feature_cols].values.astype('float32')
            if len(X)==0: continue
            test_embeds[uid] = model.embed(torch.from_numpy(X).to(DEVICE)).cpu().numpy()

    rows = []
    with torch.no_grad():
        for uid, tr in train_df.groupby('participant'):
            X_enroll = tr[feature_cols].values.astype('float32')
            if len(X_enroll) < 2: continue
            Z_enroll = model.embed(torch.from_numpy(X_enroll).to(DEVICE)).cpu().numpy()
            template = Z_enroll.mean(axis=0, keepdims=True)
            d_enroll = np.linalg.norm(Z_enroll - template, axis=1)
            tau = float(np.clip(d_enroll.mean() + 3.0*d_enroll.std(), 0.0, 2.0))

            Z_pos = test_embeds.get(uid)
            if Z_pos is None or len(Z_pos)==0: continue
            d_pos = np.linalg.norm(Z_pos - template, axis=1)

            others = [Z for u2, Z in test_embeds.items() if u2 != uid and len(Z)>0]
            if not others: continue
            pool = np.concatenate(others, axis=0)
            sel = rng.choice(len(pool), size=len(d_pos), replace=len(pool)<len(d_pos))
            Z_neg = pool[sel]
            d_neg = np.linalg.norm(Z_neg - template, axis=1)

            y_true = np.r_[np.ones_like(d_pos), np.zeros_like(d_neg)]
            y_pred = np.r_[(d_pos <= tau).astype(int), (d_neg <= tau).astype(int)]

            P, R, F1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
            FAR = ((y_pred==1)&(y_true==0)).mean()
            FRR = ((y_pred==0)&(y_true==1)).mean()
            rows.append((uid, P, R, F1, FAR, FRR, tau))

    out = pd.DataFrame(rows, columns=['user','Precision','Recall','F1','FAR','FRR','tau'])
    macro = out[['Precision','Recall','F1','FAR','FRR']].mean().to_dict()
    return out, macro

per_user, macro = evaluate_per_user_binary(model, train_df, test_df, digraph_cols)
print(per_user.head())
print("\n=== Macro means over users ===")
for k,v in macro.items(): print(f"{k}: {v:.4f}")

   user  Precision    Recall        F1       FAR       FRR       tau
0  p001   1.000000  0.933333  0.965517  0.000000  0.033333  0.520032
1  p002   0.877551  0.716667  0.788991  0.050000  0.141667  0.669773
2  p003   0.935484  0.966667  0.950820  0.033333  0.016667  0.609647
3  p004   0.812500  0.866667  0.838710  0.100000  0.066667  0.696776
4  p005   0.702381  0.983333  0.819444  0.208333  0.008333  0.902098

=== Macro means over users ===
Precision: 0.8464
Recall: 0.9248
F1: 0.8786
FAR: 0.0944
FRR: 0.0376
